# Tremor analysis

This tutorial shows how to run the tremor pipeline on prepared IMU or gyroscope data to obtain aggregated tremor measures.

## Load example data

Example IMU data (8 minutes) from a participant of the Personalized Parkinson Project is loaded. The data was prepared as explained in `data_preparation.ipynb`. The prepared data contains both accelerometer and gyroscope data, but only gyroscope data is necessary for running the tremor pipeline.

In [ ]:
from pathlib import Path
from paradigma.util import load_tsdf_dataframe

path_to_data =  Path('../../tests/data')
path_to_assets = Path('../../src/paradigma/assets')
path_to_prepared_data = path_to_data / '1.prepared_data' / 'imu'
df_data, metadata_time, metadata_values = load_tsdf_dataframe(path_to_prepared_data, prefix='IMU')

df_data

## Step 1: Preprocess data

IMU sensors collect data at a fixed sampling frequency, but the sampling rate is not uniform, causing variation in time differences between timestamps. The `preprocess_imu_data` function therefore resamples the timestamps to be uniformly distributed, and then interpolates IMU values at these new timestamps using the original timestamps and corresponding IMU values. By setting `sensor` to 'gyroscope', only gyroscope data is preprocessed and the accelerometer data is removed from the dataframe.

In [ ]:
from paradigma.config import IMUConfig
from paradigma.preprocessing import preprocess_imu_data

config = IMUConfig()
print(f'The data is resampled to {config.sampling_frequency} Hz.')

df_preprocessed_data = preprocess_imu_data(df_data, config, sensor='gyroscope')

df_preprocessed_data

## Step 2: Extract tremor features

The preprocessed gyroscope data is windowed using non-overlapping windows of length `config.window_length_s` such that features can be extracted. Features extracted from these windows are 12 mel-frequency cepstral coefficients (MFCCs), frequency of the peak in the power spectral density, power in lower frequencies (0.5 -- 3 Hz), and power around the tremor peak. The latter is not used for tremor detection, but to compute aggregate measures of tremor power in Step 4.

In [ ]:
from paradigma.config import TremorFeatureExtractionConfig
from paradigma.tremor.tremor_analysis import extract_tremor_features

config = TremorFeatureExtractionConfig()
print(f'The window length is {config.window_length_s} seconds')

df_features = extract_tremor_features(df_preprocessed_data, config)

df_features

## Step 3: Detect tremor

A pretrained logistic regression classifier is used to predict the tremor probability (`pred_tremor_proba`) for each window, based on the MFCCs. Using the prespecified threshold, a tremor label of 0 (no tremor) or 1 (tremor) is assigned (`pred_tremor_logreg`). Next, two extra criteria for rest tremor are applied. First, the frequency of the peak should be between 3-7 Hz. Second, the arm should be in rest or stable posture, defined as low frequency power below `config.movement_threshold` (established by splitting the low frequency power values measured during real-life into two clusters). The final tremor label is saved in `pred_tremor_checked`. A label for predicted arm at rest (`pred_arm_at_rest`, which is 1 when at rest and 0 when not at rest) was also saved, to control for the amount of arm movement during the observed time period when aggregating the amount of tremor time in Step 4 (if a person is moving their arm, they cannot have rest tremor).

In [ ]:
from paradigma.config import TremorDetectionConfig
from paradigma.tremor.tremor_analysis import detect_tremor

config = TremorDetectionConfig()
print(f'A threshold of {config.movement_threshold} deg\u00b2/s\u00b2 is used to determine whether the arm is at rest or in stable posture.')

tremor_detection_classifier_package_filename = 'tremor_detection_clf_package.pkl'
full_path_to_classifier_package = path_to_assets / tremor_detection_classifier_package_filename
df_predictions = detect_tremor(df_features, config, full_path_to_classifier_package)

df_predictions[['time','pred_tremor_proba','pred_tremor_logreg','pred_arm_at_rest','pred_tremor_checked']]

## Step 4: Compute aggregated tremor measures

The final step is to compute the amount of tremor time and tremor power, aggregated over all windows in the input dataframe.
Tremor time is calculated as the number of detected tremor windows, as percentage of the number of windows 
while the arm is at rest or in stable posture (when the low frequency power is below `config.movement_threshold`). This way the tremor time is controlled for the amount of time the arm is at rest or in stable posture, when rest tremor and re-emergent tremor could occur. For tremor power the following aggregates are derived: the mode, median and 90th percentile of tremor power (the last two are specified in `config.aggregates_tremor_power`). The median and modal tremor power reflect the typical tremor severity, whereas the 90th percentile reflects the maximal tremor severity within the observed timeframe. The aggregated tremor measures and metadata are stored in a json file.

In [ ]:
import pprint
from paradigma.config import TremorAggregationConfig
from paradigma.tremor.tremor_analysis import aggregate_tremor

config = TremorAggregationConfig()
d_tremor_aggregates = aggregate_tremor(df_predictions, config)

pprint.pprint(d_tremor_aggregates)